# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import (
  FileReadTool, # 
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool() # surf internet
scrape_tool = ScrapeWebsiteTool() # website content
read_resume = FileReadTool(file_path='./fake_resume.md') # read file
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md') # perform RAG 

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  3.88it/s]


- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [5]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

In [6]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [7]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [8]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [9]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [10]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True # parallel
)

In [11]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True # 2 agents working in parallel
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [12]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    ## wait for both task finishes
    
    agent=resume_strategist
)

In [13]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    ## wait for both task finishes
    
    agent=interview_preparer
)


## Creating the Crew

In [14]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [15]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [16]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his abili

Final Answer: 
- Proven experience as a Full Stack Developer or similar role
- Experience developing desktop and mobile applications
- Familiarity with common stacks
- Knowledge of multiple front-end languages and libraries (e.g. HTML/ CSS, JavaScript, XML, jQuery)
- Knowledge of multiple object-oriented languages (e.g. Java, Python) and multiple frameworks (e.g. Angular, React, Node.js)
- Familiarity with databases (e.g. MySQL, MongoDB), web servers (e.g. Apache) and UI/UX design
- Excellent communication skills and the ability to have in-depth technical discussions with both the engineering team and business people
- Excellent English language skills
- Self-starter and comfort working in an early-stage environment
- Strong project management and organizational skills
- BSc in Computer Science, Mathematics or similar field; Master’s or PhD degree is a plus
- Understanding of AI/ML models
- Understanding of big data tools, including Hadoop, Spark, Kafka, etc.
- Proficiency in HTML, CSS

Final Answer:
# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Summary
Noah Williams is a distinguished Software Engineering Leader with an 18-year tenure in the technology industry, specializing in managing remote and in-office teams, and expert in multiple programming languages and frameworks. His experience in software development, process innovation, and team collaboration make him an ideal candidate for leadership roles that require a strategic and innovative approach.

## Skills
- Proficient in Python, Ruby, and Elixir programming languages
- Experienced in developing and managing AI and data science projects
- Skilled in leading and coordinating remote and in-office teams
- Expertise in framework development and implementation
- Knowledge of multiple front-end languages and libraries (e.g. HTML/ CSS, JavaScript, XML, jQuery)
- Knowledge of multiple object-oriented languages (e.g. Java, Python) and multiple frameworks (e.g. Angular, React, Node.js)

Final Answer:
Based on Noah Williams' extensive experience and skills highlighted in his resume, here are some potential interview questions and talking points that can be used to assess his fit for the role:

1. Experience and Expertise:
- Can you walk us through your experience as a Software Engineering Leader, focusing on your roles in managing remote and in-office teams?
- How have you utilized your proficiency in Python, Ruby, and Elixir programming languages in developing and managing AI and data science projects?
- Can you provide examples of how you have led and coordinated remote and in-office teams effectively, especially in the context of framework development and implementation?

2. Project Experiences:
- Tell us more about your role as the Founder of crewAI and the development of the framework for orchestrating role-playing autonomous AI agents.
- How have your contributions to open-source projects such as machinery, active_model_serializers, gioco, sigma, and keeper impac

- Dislplay the generated `tailored_resume.md` file.

In [17]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Summary
Noah Williams is a distinguished Software Engineering Leader with an 18-year tenure in the technology industry, specializing in managing remote and in-office teams, and expert in multiple programming languages and frameworks. His experience in software development, process innovation, and team collaboration make him an ideal candidate for leadership roles that require a strategic and innovative approach.

## Skills
- Proficient in Python, Ruby, and Elixir programming languages
- Experienced in developing and managing AI and data science projects
- Skilled in leading and coordinating remote and in-office teams
- Expertise in framework development and implementation
- Knowledge of multiple front-end languages and libraries (e.g. HTML/ CSS, JavaScript, XML, jQuery)
- Knowledge of multiple object-oriented languages (e.g. Java, Python) and multiple frameworks (e.g. Angular, React, Node.js)
- Familiarity with databases (e.g. MySQL, MongoDB), web servers (e.g. Apache) and UI/UX design
- Strong project management and organizational skills
- Understanding of AI/ML models
- Understanding of big data tools, including Hadoop, Spark, Kafka, etc.
- Proficiency in HTML, CSS and JavaScript
- Experience with consumer applications and data handling
- Familiarity with data privacy regulations and best practices

## Work Experience

### Director of Software Engineering at DataKernel (2022 - Present)
- Transformed the engineering division into a key revenue pillar for DataKernel
- Spearheaded the integration of cutting-edge AI technologies and scalable vector databases
- Led the team to substantial skill development and strategic project achievements

### Senior Software Engineering Manager at DataKernel (2019 - 2022)
- Directed engineering strategy and operations in collaboration with C-level executives
- Managed diverse teams across multiple time zones, fostering transparency and mutual respect

### Founder & CEO of InnovPet (2019 - 2022)
- Founded a startup focused on innovative IoT solutions for pet care
- Oversaw product development, establishing successful market entry and funding

### Engineering Manager at EliteDevs (2018 - 2019)
- Formulated and executed strategic plans, enhancing inter-departmental coordination
- Managed multiple engineering teams, fostering a culture of productivity and innovation

### Engineering Manager at PrintPack (2016 - 2018)
- Led a high-performance engineering team, increasing company revenue by 500%
- Integrated data analytics into business decision-making processes

### Senior Software Engineer at DriveAI (2015 - 2016)
- Developed and optimized a central API, improving system performance and user satisfaction
- Implemented critical enhancements, including advanced caching strategies

### CTO at BetCraft (2013 - 2015)
- Led technology strategy post-Series A funding, guiding the company through technological advancement
- Improved platform performance and expanded market reach

## Education
- MBA in Information Technology from London Business School
- Advanced Leadership Techniques Certification from University of London
- Data Science Specialization Certification from Coursera (Johns Hopkins University)
- B.Sc. in Computer Science from University of Edinburgh

Noah Williams is a valuable asset for companies seeking a strategic and innovative leader with a strong technical background in software engineering, AI, and data science.

- Dislplay the generated `interview_materials.md` file.

In [18]:
display(Markdown("./interview_materials.md"))

Based on Noah Williams' extensive experience and skills highlighted in his resume, here are some potential interview questions and talking points that can be used to assess his fit for the role:

1. Experience and Expertise:
- Can you walk us through your experience as a Software Engineering Leader, focusing on your roles in managing remote and in-office teams?
- How have you utilized your proficiency in Python, Ruby, and Elixir programming languages in developing and managing AI and data science projects?
- Can you provide examples of how you have led and coordinated remote and in-office teams effectively, especially in the context of framework development and implementation?

2. Project Experiences:
- Tell us more about your role as the Founder of crewAI and the development of the framework for orchestrating role-playing autonomous AI agents.
- How have your contributions to open-source projects such as machinery, active_model_serializers, gioco, sigma, and keeper impacted your approach to software development and collaboration?
- Share with us the challenges you faced and the successes you achieved while leading the development of gamification solutions for Ruby on Rails applications.

3. Contributions and Interests:
- How do you actively engage with the developer community as a member of the Developer Program, and what motivates you to support junior developers in contributing to open-source projects?
- What innovative technologies and frameworks are you currently exploring, and how do you approach continuous learning and skill development in the tech industry?
- Discuss the importance of collaborative intelligence and seamless agent interaction in AI projects, and how you advocate for these principles in your work.

4. Communication Style and Leadership:
- How do you ensure clear and concise communication of ideas and project requirements within your team and across departments?
- Share examples of how you foster collaboration and support in team interactions, and how you approach problem-solving and decision-making in a strategic and innovative manner.
- Describe a situation where your strategic leadership approach has driven growth and innovation in a project or team setting.

These questions and talking points aim to delve deeper into Noah's background, skills, and experiences, allowing the interviewer to assess his suitability for leadership roles that require a strategic and innovative approach in the tech industry.

# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)